In [ ]:
import numpy as np
import torch
import random
from PIL import Image
import cv2
import time
import scipy
import torch
import torch.nn as nn
from torch import optim
from sklearn.cluster import KMeans

In [ ]:
class Catch():

    def __init__(self, rows = 10, cols = 5): 

        self.max_steps = 100
        self.n_steps = 0
        self.n_actions = 3
        self.n_states = 250
        self.rows = rows
        self.cols = cols
        self.colors = {0: (0,0,0),           #blank(object id = 0)
                        1: (0, 255, 0),      #ball(object id = 2)
                        2: (255, 0, 0)       #agent(object id = 3)
                        } 
        self.env = np.zeros([self.rows,self.cols])
        self.place_goal()
        self.place_agent()

    def place_goal(self):
        
        self.gx = 0
        self.gy = np.random.randint(0,5)
        self.env[self.gx,self.gy] = 1
    
    def place_agent(self):
        
        self.ax = self.rows-1
        self.ay = np.random.randint(0,5)
        self.env[self.ax,self.ay] = 2
    
    def step(self, action):

        terminal = False

        if action == 0:
            reward = self.move(y=1)          #EAST
        elif action == 1:
            reward = self.move(y=-1)         #WEST 
        elif action == 2:                
            reward = self.move(y=0)          #NOTHING
        
        if self.n_steps >= 100:
            terminal = True

        return (self.ay*50+self.gx*5+self.gy),reward,terminal

    def move(self,y):

        reward = 0
        terminal = False
        next_y = self.ay + y

        if next_y == -1 or next_y == self.cols:
            pass
        else:
            self.clear_block(self.ax,self.ay)
            self.ay = next_y
            self.place_block(2,self.ax,self.ay) 
        
        if(self.gx==self.rows-1):
            #If the ball is at the bottom most row
            self.clear_block(self.gx,self.gy)
            self.place_goal()
        else:
            self.clear_block(self.gx,self.gy)
            self.gx += 1
            self.place_block(1,self.gx,self.gy) 

            if(self.gx==self.rows-1):
                if(self.gy==self.ay):
                    reward = 1
            

        self.n_steps += 1
        return reward

    def clear_block(self,x,y):
        
        self.env[x,y] = 0

    def place_block(self,obj,x,y):
        
        self.env[x,y] = obj

    def reset(self):
        
        self.n_steps = 0
        self.env = np.zeros([self.rows,self.cols])
        self.place_goal()
        self.place_agent()
        return (self.ay*50+self.gx*5+self.gy)
    
    def render(self, render_time=100):
        #cv2.destroyAllWindows()
        img = np.zeros((self.rows*31, self.cols*31, 3), dtype=np.uint8)

        for i in range(self.rows):
            for j in range(self.cols):
                obj = self.env[i,j]
                img = self.fill(img,i,j,obj)

        img = Image.fromarray(img, 'RGB')
        cv2.imshow("image", np.array(img))
        cv2.waitKey(render_time)

    def fill(self, m, x, y, c, s=31):

        t = (s-1)//2
        x_t = (x)*s + t
        y_t = (y)*s + t
        m[x_t-t:x_t+t,y_t-t:y_t+t] = self.colors[c]
        return m

    def sample_action(self):

        return np.random.randint(0, self.n_actions)
    
    def state_features(self):
        state = np.zeros([250,4],dtype=np.float32)
        for i in range(250):
            state[i,0] = 9
            state[i,1] = i//50
            state[i,2] = ((i//5)%10)
            state[i,3] = i%5
        return state

In [ ]:
class Memory():
    def __init__(self,env,size):
        
        self.size = size
        self.env = env
        self.state_memory = np.zeros([size],dtype = np.int)
        self.next_state_memory = np.zeros([size],dtype = np.int)
        self.action_memory = np.zeros(size,dtype=np.int)
        self.reward_memory = np.zeros(size,dtype=np.float32)
        self.terminal_memory = np.zeros(size,dtype=np.int8)
        self.memory_count = 0
    
    def store(self,state,action,reward,next_state,terminal):
        
        index = self.memory_count%self.size

        self.state_memory[index] = state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.next_state_memory[index] = next_state
        self.terminal_memory[index] = terminal
        self.memory_count+=1 
    
    def to_tensor(self):
        self.state_memory = torch.tensor(self.state_memory).to(device)
        self.next_state_memory = torch.tensor(self.next_state_memory).to(device)
        self.reward_memory = torch.tensor(self.reward_memory).to(device)

    def store_random(self):
        state = self.env.reset()
        done = False
        for i in range(self.size):
            action = np.random.randint(0,self.env.n_actions)
            next_state,reward,done = self.env.step(action)
            self.store(state,action,reward,next_state,done)
            if done:
                state = self.env.reset()
                done = False
            else:
                state = next_state


In [ ]:
def rank_constrained(F_D,F_K):
    m = torch.nn.Softmax(dim=2)
    P = torch.matmul(m(F_D),m(F_K))
    return P

In [ ]:
device = torch.device("cuda:0")
learning_rate = 5e-3
total_iters = 1000000
gamma = 0.99
mem_size = 1000000
d = 50
rank = 150

In [ ]:
env = Catch()
features = env.state_features()
mem = Memory(env,mem_size)
mem.store_random()
data = features[mem.state_memory]

In [ ]:
km = KMeans(
  n_clusters=d, init='k-means++',
  n_init=10,tol=1e-04)

In [ ]:
km.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
ph_index = km.predict(features)
ph_feature = np.eye(d)[ph_index]

In [ ]:
#Estimating reward and transition matrix
Reward = torch.zeros(env.n_states,env.n_actions)
Transition = torch.zeros(env.n_actions,env.n_states,env.n_states)
for i in range(1000000):
    s,a,r,n_s = mem.state_memory[i],mem.action_memory[i],mem.reward_memory[i],mem.next_state_memory[i]
    Reward[s,a] += r
    Transition[a,s,n_s] += 1
z = torch.unsqueeze(torch.sum(Transition,axis=2),dim=2)
P_A = torch.divide(Transition,z) #unbiased transition matrix
R_A = torch.divide(Reward,torch.sum(Transition,axis=2).T) #unbiased reward matrix

In [ ]:
v_basis = torch.tensor(ph_feature, device = device)
P_A = P_A.to(device)
F_D = (torch.rand(size=[env.n_actions,env.n_states,rank],device=device)*2 - 1).requires_grad_(True)
F_K = (torch.rand(size=[env.n_actions,rank,env.n_states],device=device)*2 - 1).requires_grad_(True)

In [ ]:
#MLE 
for i in range(total_iters):
    P = rank_constrained(F_D,F_K)
    loss = torch.sum(torch.multiply(-P_A,torch.log(P)))
    loss.backward()
    with torch.no_grad():
        F_D -= learning_rate * F_D.grad 
        F_K -= learning_rate * F_K.grad 
    F_D.grad = None
    F_K.grad = None
    if i%5000 == 0:
        print("i = ", i)
        print("loss = ", loss)

In [ ]:
#VE
T = torch.matmul(P_A,v_basis.float())
for i in range(total_iters):
    P = rank_constrained(F_D,F_K)
    loss = torch.sum(torch.linalg.norm(T-torch.matmul(P,v_basis.float()),dim=2)**2)
    loss.backward()
    with torch.no_grad():
        F_D -= learning_rate * F_D.grad 
        F_K -= learning_rate * F_K.grad
    F_D.grad = None
    F_K.grad = None
    if i%5000 == 0:
        print("i = ", i)
        print("loss = ", loss)

EVAlUATE USING LSTD

In [ ]:
def collect_traj(size,policy,P_es,R_es):
    state = np.zeros(size,dtype=np.int)
    next_state = np.zeros(size,dtype=np.int)
    reward = np.zeros(size,dtype=np.int)
    action = np.zeros(size,dtype=np.int)
    term = np.zeros(size,dtype=np.int)
    s = env.reset()
    
    #Storing on-policy data
    for j in range(size):
        
        a = policy[s]
        n_s,r,t = env.step(a)
        n_s_hat = np.random.choice(state_list, 1, p=P_es[a,s])
        #n_s_hat = np.argmax(P_es[a,s])
        r_hat = R_es[s,a]
        state[j] = s
        action[j] = a
        next_state[j] = n_s_hat
        reward[j] = r_hat
        term[j] = t
        
        if t:
            s = env.reset()
        else:
            s = n_s
    return state,next_state,reward,action,term

In [ ]:
def value_iteration(env,t,r,gamma=0.99,epsilon=0.01):
    v = np.zeros(env.n_states)
    v_new = np.zeros(env.n_states)
    t = np.array(t.to('cpu').detach())
    r = np.array(r.to('cpu').detach())
    while True:
        for s in range(env.n_states):
            v_temp = np.zeros(env.n_actions)
            for a in range(env.n_actions):
                v_temp[a] = r[s,a] + gamma*np.dot(t[a,s,:],v) 
            v_new[s] = np.max(v_temp)
        if np.max(np.abs(v - v_new)) < epsilon:
            break
        v = np.copy(v_new)
    return v

In [ ]:
def policy_eval(policies,env,t,r,gamma,epsilon=0.01):
    v_pi = []
    t = np.array(t.to('cpu'))
    r = np.array(r.to('cpu'))
    for policy in policies:
        v = np.zeros(env.n_states)
        v_new = np.zeros(env.n_states)
        while True:
            for s in range(env.n_states):
                a = int(policy[s])
                v_new[s] = r[s,a] + gamma*np.dot(t[a,s,:],v)
            if np.max(np.abs(v - v_new)) < epsilon:
                break
            v = np.copy(v_new)
        v_pi.append(v)
    return v_pi

In [ ]:
def LSTD(state,next_state,reward,action,term,feature,d,gamma=1):
    A = np.eye(d)*0.001 
    b = np.zeros(d)
    for i in range(10000):
        s = state[i]
        a = action[i]
        r = reward[i]
        n_s = next_state[i]
        t = term[i]
        
        x = feature[s]
        xp = feature[n_s]  
        
        A += np.outer(x, (x - gamma*xp))
        b += x * r
    w = np.dot(np.linalg.pinv(A), b)
    return w

In [ ]:
#Approx Policy iteration with LSTD
state_list = np.arange(env.n_states)
policy = np.random.randint(0,env.n_actions,(env.n_states))
P_est = P.detach().cpu().numpy()
R_est = R_A.detach().cpu().numpy()
for i in range(40): 
    s,ns,r,a,t = collect_traj(10000,policy,P_est,R_est)
    w = LSTD(s,ns,r,a,t,ph_feature,50)
    for j in range(env.n_states):
        v_temp = np.zeros(env.n_actions)
        for k in range(env.n_actions):
            n_j = np.random.choice(state_list, 1, p=P_est[k,j])
            v_temp[k] = R_est[j,k] + gamma*np.dot(ph_feature[n_j],w)
        policy[j] = np.argmax(v_temp)
    v_cur = policy_eval([policy],env,P_A,R_A,gamma=0.99)
    print(np.mean(v_cur))

EVALUATE USING DDQN

In [ ]:
class DQN(nn.Module):
    
    def __init__(self,learning_rate,d,n_actions,epsilon):
        
        super(DQN, self).__init__()

        self.epsilon = epsilon
        self.n_actions = n_actions
        self.d = d
        
        self.linear = nn.Linear(d,n_actions,bias=False)
        nn.init.normal_(self.linear.weight,0.0,1/np.sqrt(d))

        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters(),lr = learning_rate)

    def forward(self, x):
        
        q_value = self.linear(x)        
        return q_value
    
    def action(self,x):
        x = torch.tensor(x).to(device)
        r = np.random.random()
        if r<self.epsilon:
            action = np.random.randint(0,self.n_actions)
        else:
            with torch.no_grad():
                q_val = self.forward(x.float())
                action = torch.argmax(q_val).item()
        return action

In [ ]:
n_actions = env.n_actions

DQN_lr = 1e-3
n_actions = env.n_actions
epsilon = 0.01


batch_size = 32
num_learn_steps = 1000000
num_learn_freq = 4
dqn_gamma = 0.99
target_update = 2000
P_es = P.detach().cpu().numpy()
R_es = R_A.detach().cpu().numpy()

In [ ]:
Q_model = DQN(DQN_lr,d,n_actions,epsilon).to(device)
Q_model_target = DQN(DQN_lr,d,n_actions,epsilon).to(device)
Q_model_target.load_state_dict(Q_model.state_dict())

<All keys matched successfully>

In [ ]:
replay_buffer = Memory(env,10000)

In [ ]:
batch_index = np.arange(batch_size)
done = False
s = env.reset()
score = 0
scores = []
best_avg = 0
for n in range(1,num_learn_steps+1):
    a = Q_model.action(ph_feature[s])
    ns,r,done = env.step(a)
    score += r
    ns_hat = np.random.choice(state_list, 1, p=P_es[a,s])
    r_hat = R_es[s,a]

    replay_buffer.store(s,a,r_hat,ns_hat,done)
    if done:
      scores.append(score)
      s = env.reset()
      done = False
      score = 0
    else:
      s = ns
    
    if replay_buffer.memory_count < mem_size:
        m = replay_buffer.memory_count
    else:
        m = mem_size

    if replay_buffer.memory_count > batch_size and n%num_learn_freq==0 :

        batch = np.random.choice(10000, batch_size, replace=False)
        s_batch = torch.tensor(ph_feature[replay_buffer.state_memory[batch]]).to(device)
        ns_batch = torch.tensor(ph_feature[replay_buffer.next_state_memory[batch]]).to(device)
        r_batch = torch.tensor(replay_buffer.reward_memory[batch],dtype=torch.float32).to(device)
        a_batch = torch.tensor(replay_buffer.action_memory[batch])
        term_batch = torch.tensor(replay_buffer.terminal_memory[batch],dtype=torch.int8).to(device)
        
        q_val = Q_model.forward(s_batch.float())[batch_index,a_batch]
        max_a = torch.argmax(Q_model.forward(ns_batch.float()),dim=1).detach()
        next_q_val = Q_model_target(ns_batch.float())[batch_index,max_a].detach()

        q_target = r_batch + dqn_gamma*next_q_val*(1-term_batch) 
        loss = Q_model.criterion(q_val,q_target).to(device)
        
        Q_model.optimizer.zero_grad()
        loss.backward()
        Q_model.optimizer.step()
        Q_model.optimizer.zero_grad()
    
    if n%500 == 0:
      print("i = ", n)
      print('score now', np.mean(scores))

    if n%target_update == 0:
        Q_model_target.load_state_dict(Q_model.state_dict())

    if len(scores) == 100:
        avg = np.mean(scores)
        if avg > best_avg:
            best_avg = avg
            torch.save(Q_model.state_dict(),'best.pt')
        scores.pop(0)
best_avg 

In [ ]:
Q_model.load_state_dict(torch.load('best.pt'))

<All keys matched successfully>

In [ ]:
done = False
score = 0
scores = []

for i in range(100): 
  s = env.reset()
  done = False
  score = 0
  while not done:
    s_ = torch.tensor(ph_feature[s]).to(device)
    q_val = Q_model(s_.float())
    a = torch.argmax(q_val).item()
    ns,r,done = env.step(a)
    score += r
    s = ns
  scores.append(score)
print(np.mean(scores))